## Comparison between datasets
### Part two:    Word Associations  
  
An established method for uncovering hidden bias is to use the **Implicit Association Test (IAT)**. The test is administered by showing the subject a series of words relating to a specific concept, and asking them to sort them into categories that are paired with a specific attribute. For example, they may be shown types of flowers and types of insects, and asked to categorize them into pairings that include either pleasant words or unpleasant words. By pairing the target concepts (flowers and insects) with the attribute words (pleasant and unpleasant) and measuring the response time to sort pairings, it is possible to uncover hidden attitudes. In this example, it is almost always the case that people can sort words into categories more quickly when flowers are paired with pleasant words, and insects are paired with unpleasant words. Response times slow dramatically when the pairings are reversed, so that it takes much longer to determine the correct category of "cockroach" if the options are pleasant/insect or unpleasant/flower. Read more [here](https://implicit.harvard.edu/implicit/education.html) and **take the test [here](https://implicit.harvard.edu/implicit/takeatest.html)**.  
  
In a [paper](../Papers/Caliskan%202017%20-%20Semantics%20derived%20automatically%20from%20language%20corpora%20contain%20human-like%20biases.pdf) from 2017, Caliskan and her team showed that commonly observed biases in the IAT test could also be observed in gloVe word embeddings, by looking at cosine similarity between vector pairs.

Depending upon the dataset chosen, gloVe embeddings are derived from Wikipedia, Twitter or Common Crawl, and therefore reflect broadly occuring bias among a vast range of authors. In this exercise, we will use the WEAT (word embedding association test) developed by Caliskan, but apply it to specific domains to see whether more narrowly held biases are revealed. 

In [1]:
import gensim
import numpy as np
import pandas as pd

from gensim.models import KeyedVectors

In [2]:
# Load the embedding vectors that were just calculated
female_vectors = KeyedVectors.load('../Data/female_model.wv', mmap='r')
male_vectors = KeyedVectors.load('../Data/male_model.wv', mmap='r')
movie_vectors = KeyedVectors.load('../Data/movie_model.wv', mmap='r')
lyrics_vectors = KeyedVectors.load('../Data/lyrics_model.wv', mmap='r')

# Set up lists of words used in IAT
flowers = ['hyacinth', 'marigold', 'poppy', 'azalea', 'crocus', 'iris', 'orchid', 'rose', 'bluebell', 'daffodil', 'lilac', 'pansy', 'tulip', 'buttercup', 'daisy', 'lily', 'peony', 'violet', 'carnation', 'gladiola', 'magnolia', 'petunia', 'zinnia']
insects = ['ant', 'caterpillar', 'flea', 'locust', 'spider', 'bedbug', 'centipede', 'maggot', 'tarantula', 'bee', 'cockroach', 'gnat', 'mosquito', 'termite', 'cricket', 'hornet', 'moth', 'wasp', 'blackfly', 'dragonfly', 'horsefly', 'roach', 'weevil']
pleasant = ['caress', 'freedom', 'health', 'love', 'peace', 'cheer', 'friend', 'heaven', 'loyal', 'pleasure', 'diamond', 'gentle', 'honest', 'lucky', 'rainbow', 'diploma', 'gift', 'honor', 'miracle', 'sunrise', 'family', 'happy', 'laughter', 'paradise', 'vacation']
unpleasant = ['abuse', 'crash', 'filth', 'murder', 'sickness', 'accident','death', 'grief', 'poison', 'stink', 'assault', 'disaster', 'hatred', 'pollute', 'tragedy','bomb', 'divorce', 'jail', 'poverty', 'ugly', 'cancer', 'evil', 'kill', 'rotten', 'vomit','agony', 'prison']
instruments = ['bagpipe','cello','guitar','lute','trombone','banjo','clarinet','harmonica','mandolin','trumpet','bassoon','drum','harp','oboe','tuba','bell','fiddle','harpsichord','piano','viola','bongo','flute','horn','saxophone','violin']
weapons = ['arrow', 'club', 'gun', 'missile', 'spear', 'axe', 'dagger', 'harpoon', 'pistol', 'sword', 'blade', 'dynamite', 'hatchet', 'rifle', 'tank', 'bomb', 'firearm', 'knife', 'shotgun', 'teargas', 'cannon', 'grenade', 'mace', 'slingshot', 'whip']
male_names = ['john', 'paul', 'mike', 'kevin', 'steve', 'greg', 'jeff', 'bill']
female_names = ['amy', 'joan', 'lisa', 'sarah', 'diana', 'kate', 'ann', 'donna']
career = ['executive', 'management', 'professional', 'corporation', 'salary', 'office', 'business', 'career']
family = ['home', 'parents', 'children', 'family', 'cousins', 'marriage', 'wedding', 'relatives']
maths = ['math', 'algebra', 'geometry', 'calculus', 'equations', 'computation', 'numbers', 'addition']
science = ['science', 'technology', 'physics', 'chemistry', 'einstein', 'nasa', 'experiment', 'astronomy']
arts = ['poetry', 'art', 'dance', 'literature', 'novel', 'symphony', 'drama', 'sculpture']
male_terms = ['male', 'man', 'boy', 'brother', 'he', 'him', 'his', 'son']
female_terms = ['female', 'woman', 'girl', 'sister', 'she', 'her', 'hers', 'daughter']
mental_disease = ['sad', 'hopeless', 'gloomy', 'tearful', 'miserable', 'depressed']
physical_disease = ['sick', 'illness', 'influenza', 'disease', 'virus', 'cancer']
temporary = ['impermanent', 'unstable', 'variable', 'fleeting', 'short-term', 'brief', 'occasional']
permanent = ['stable', 'always', 'constant', 'persistent', 'chronic', 'prolonged', 'forever']
freedom = ['capitalism', 'freedom', 'autonomy', 'independence', 'liberty', 'sovereignty', 'license', 'choice', 'different']
equality = ['socialism', 'equity', 'fairness', 'coequality', 'equivalence', 'parity', 'equal', 'impartiality', 'same']

The WEAT test involve looking at the cosine similarity between pairs of word embeddings.  
  
If the author holds no implicit bias, then it is reasonable to assume that there should be no difference in the cosine similarity between either set of target words and the attribute words. This is because the embeddings are derived from the written works of the author, and therefore reflect the way that they use language and the words that they choose when describing situations, characters or concepts.  
  
For example, we could test for gender bias by selecting a target set of scientific words and artistic words, and then calculating the average cosine similarity versus sets of male and female words. If the author does not associate male  with science and female with arts, then there should be no statistical difference between the two means.  
  
To determine statistical significance, we'll use **Cohen's d**, which is a standard measure of effect size when comparing two population means.  
  
Cohen's d = $\frac{\bar{x_1}-\bar{x_2}}{sd_{pooled}}$  
  
Typically effect sizes of 0.2, 0.5 and 0.8 are considered small, medium and large.

In [3]:
def cohen_d(target1, target2, attr1, attr2, model, verbose=False):
    """Calculates cosine similarity between the embeddings for two sets of target words versus two sets of 
       attribute words, takes the difference, and returns the Cohen's d statistic as a measure of effect size.
       
       Args:
           target1, target2: sets of concept words to be tested for bias
           attr1, attr2: sets of attributes representing the bias being tested for
           model: (gensim.KeyedVectors object) word embeddings to use
           verbose: optional flag to return further detail for investigating the drivers of any difference
       Returns:
           Cohen's d statistic: 0.2 is a small effect, 0.5 is medium and 0.8 is large
           Swab_x_detail, swab_y_detail (if verbose is set to True): list of tuples, showing each target word 
           and the mean association versus attribute1 and attribute2"""
    
    # First, ensure that the words are actually in the vocab
    check1 = [word for word in target1 if word in model.vocab]
    check2 = [word for word in target2 if word in model.vocab]
    check3 = [word for word in attr1 if word in model.vocab]
    check4 = [word for word in attr2 if word in model.vocab]
    
    # Then, cap the length of each pair of lists so that the two are equal
    if len(check1)>len(check2):
        check1 = check1[:len(check2)]
    else:
        check2 = check2[:len(check1)]
        
    if len(check3)>len(check4):
        check3 = check3[:len(check4)]
    else:
        check4 = check4[:len(check3)]
    
    # Revise inputs so that they contain only words in the vocab and they are symmetrical
    target1, target2, attr1, attr2 = check1, check2, check3, check4
    
    # Calc difference in mean similarity vs each set of attributes for target group 1
    swab_x = []
    swab_x_detail = []
    for word in target1:
        cos_sim_a = [model.similarity(word,i) for i in attr1]
        cos_sim_b = [model.similarity(word,j) for j in attr2]
        diff = np.mean(cos_sim_a)-np.mean(cos_sim_b)
        swab_x.append(diff)
        swab_x_detail.append((word, np.mean(cos_sim_a), np.mean(cos_sim_b)))
    
    # Calc difference in mean similarity vs each set of attributes for target group 2
    swab_y = []
    swab_y_detail = []
    for word in target2:
        cos_sim_a = [model.similarity(word,i) for i in attr1]
        cos_sim_b = [model.similarity(word,j) for j in attr2]
        diff = np.mean(cos_sim_a)-np.mean(cos_sim_b)
        swab_y.append(diff)
        swab_y_detail.append((word, np.mean(cos_sim_a), np.mean(cos_sim_b)))
    
    # Calculate difference in mean similarity for the pooled group of target words
    # vs each set of attributes. The standard deviation of all differences will be 
    # used as a scaling factor when calculating cohen's d statistic
    swab_z = []
    for word in target1+target2:
        cos_sim_a = [model.similarity(word,i) for i in attr1]
        cos_sim_b = [model.similarity(word,j) for j in attr2]
        diff = np.mean(cos_sim_a)-np.mean(cos_sim_b)
        swab_z.append(diff)
    
    # Cohen's d = difference in means divided by th epooled standard deviation
    mean_diff = np.mean(swab_x)-np.mean(swab_y)
    std = np.std(swab_z)
    
    if verbose:
        return mean_diff/std, swab_x_detail, swab_y_detail
    else:
        return mean_diff/std

def view_detail(target1, target2, attr1, attr2, model):
    """Runs verbose version of Cohen D function above, and returns dataframe
       showing detail of net scores
       
       Args:target1, target2: sets of concept words to be tested for bias
            attr1, attr2: sets of attributes representing the bias being tested for
            model: (gensim.KeyedVectors object) word embeddings to use
       Returns: Pandas dataframe containing each word, the average similarity versus
            attr1 (labelled A), attr2 (labelled B) and the net score
         """
    score,a,b = cohen_d(target1, target2, attr1, attr2, model, True)
    df = pd.DataFrame(a,b)
    # turn index (tuple of word, value1, value2) into a column that we'll call temp
    df.reset_index(level=0, inplace=True)
    
    # assign column names
    df.columns=['temp','target1', 'A1', 'B1']
    
    # split temp tuple into component columns
    df['target2'], df['A2'], df['B2'] = df['temp'].str
    
    # calculate net attribute score for each word
    df['net_A1'] = df['A1'] - df['B1']
    df['net_A2'] = df['A2'] - df['B2']
    
    # remove temp column
    df = df.drop(['temp'], axis=1)
    
    # reorder columns
    df = df[['target1', 'A1', 'B1','net_A1',
             'target2', 'A2', 'B2','net_A2']]
    
    print('Cohens d: {0:.2f}'.format(score))
    print('Average net A1: {0:.2f} \nAverage net A2: {1:.2f}'.format(np.mean(df['net_A1']), np.mean(df['net_A2'])))
    return df

### Testing  
  
The first step is to download pre-trained gloVe embeddings from the Stanford NLP site, [here](https://nlp.stanford.edu/projects/glove/), and use these to recreate Caliskan's results. This step isn't shown in this workbook, because the method is the same as below but the associated data file is very large. Instead we'll just present results using our bespoke trained embeddings and look for interesting associations. 

#### a) Flowers vs Insects    
  
This is a classic IAT test and a well established relationship, where flowers are generally seen as more pleasant than insects. We'll see how well this is reflected in our datasets;

In [4]:
print("Female Authors: {:.2f}".format(cohen_d(flowers, insects, pleasant, unpleasant, female_vectors)))
print("Male Authors: {:.2f}".format(cohen_d(flowers, insects, pleasant, unpleasant, male_vectors)))
print("Movie Scripts: {:.2f}".format(cohen_d(flowers, insects, pleasant, unpleasant, movie_vectors)))
print("Song Lyrics: {:.2f}".format(cohen_d(flowers, insects, pleasant, unpleasant, lyrics_vectors)))

Female Authors: 0.15
Male Authors: -0.01
Movie Scripts: 0.78
Song Lyrics: 0.73


The relationship is visible in all data apart from the works of male authors. If we look at the cosine similarity scores for male authors in more detail;

In [5]:
view_detail(flowers, insects, pleasant, unpleasant, male_vectors)

Cohens d: -0.01
Average net A1: -0.11 
Average net A2: -0.11


,target1,A1,B1,net_A1,target2,A2,B2,net_A2
0,hyacinth,0.262470,0.424541,-0.162071,ant,0.307444,0.477451,-0.170007
1,poppy,0.253178,0.354917,-0.101738,caterpillar,0.362725,0.476503,-0.113778
2,iris,0.320511,0.448017,-0.127506,flea,0.354805,0.434885,-0.080080
3,orchid,0.258529,0.364524,-0.105995,locust,0.271611,0.416257,-0.144646
4,rose,0.070141,0.085615,-0.015473,spider,0.304322,0.419675,-0.115352
5,bluebell,0.326930,0.447678,-0.120748,centipede,0.280367,0.319470,-0.039104
6,lilac,0.295143,0.436254,-0.141112,maggot,0.330508,0.440828,-0.110320
7,pansy,0.137049,0.281499,-0.144450,tarantula,0.169128,0.281134,-0.112006
8,tulip,0.263616,0.382331,-0.118714,bee,0.339156,0.477989,-0.138833
9,daisy,0.133131,0.126421,0.006710,gnat,0.327698,0.375513,-0.047815


The issue is that while flowers have some association with pleasant words, their association with unpleasant words is actually stronger. That means that although there is a net unpleasant score for insects (as with the other datasets) there is also a net unpleasant score for flowers, which leads to the low Cohen's d score (indicating no significant difference in the level of unpleasantness between flowers and insects)

#### b) Maths vs Arts  
  
We'll check these targets for gender bias

In [6]:
print("Female Authors: {:.2f}".format(cohen_d(maths, arts, male_terms, female_terms, female_vectors)))
print("Male Authors: {:.2f}".format(cohen_d(maths, arts, male_terms, female_terms, male_vectors)))
print("Movie Scripts: {:.2f}".format(cohen_d(maths, arts, male_terms, female_terms, movie_vectors)))
print("Song Lyrics: {:.2f}".format(cohen_d(maths, arts, male_terms, female_terms, lyrics_vectors)))

Female Authors: -0.54
Male Authors: 0.02
Movie Scripts: 0.17
Song Lyrics: -0.52


There aren't significant gender differences in the way that math or art is described in either the works of male authors or movie scripts. However, female authors and lyricists both associate female nouns and pronouns (woman, girl, she, her etc) more strongly with the arts than with mathematics.

#### c) Mental vs Physical disease  

In [8]:
print("Female Authors: {:.2f}".format(cohen_d(mental_disease, physical_disease, male_terms, female_terms, female_vectors)))
print("Male Authors: {:.2f}".format(cohen_d(mental_disease, physical_disease, male_terms, female_terms, male_vectors)))
print("Movie Scripts: {:.2f}".format(cohen_d(mental_disease, physical_disease, male_terms, female_terms, movie_vectors)))
print("Song Lyrics: {:.2f}".format(cohen_d(mental_disease, physical_disease, male_terms, female_terms, lyrics_vectors)))


Female Authors: -1.55
Male Authors: -1.46
Movie Scripts: -1.50
Song Lyrics: -0.15


All datasets (apart from lyrics) associate female terms much more strongly with mental illness than they do for male terms. Physical illness is more evenly balanced, but is slightly more likely to relate to females.  
  
We can also look at how diseases are expected to persist.

In [9]:
print("Female Authors: {:.2f}".format(cohen_d(mental_disease, physical_disease, temporary, permanent, female_vectors)))
print("Male Authors: {:.2f}".format(cohen_d(mental_disease, physical_disease, temporary, permanent, male_vectors)))
print("Movie Scripts: {:.2f}".format(cohen_d(mental_disease, physical_disease, temporary, permanent, movie_vectors)))
print("Song Lyrics: {:.2f}".format(cohen_d(mental_disease, physical_disease, temporary, permanent, lyrics_vectors)))

Female Authors: 0.83
Male Authors: -0.38
Movie Scripts: 1.17
Song Lyrics: -1.30


Female authors view words in the mental disease set as being temporary afflictions, whereas male authors show no difference in their associations between mental disease and duration. They do however show a stronger tendency to describe physical illness as permanent.

Movie scripts portray physical illness as permanent, while song lyrics strongly associate both physical and mental illness as being temporary.

In [10]:
view_detail(mental_disease, physical_disease, temporary, permanent, female_vectors)

Cohens d: 0.83
Average net A1: 0.09 
Average net A2: 0.03


,target1,A1,B1,net_A1,target2,A2,B2,net_A2
0,sad,0.330556,0.169816,0.160740,sick,0.149597,0.174408,-0.024811
1,hopeless,0.467321,0.357552,0.109769,illness,0.229543,0.264330,-0.034787
2,gloomy,0.572050,0.366900,0.205150,influenza,0.443839,0.359768,0.084070
3,tearful,0.505256,0.466448,0.038807,disease,0.510107,0.433572,0.076534
4,miserable,0.116718,0.169180,-0.052462,virus,0.429896,0.392743,0.037153
5,depressed,0.484364,0.400152,0.084213,cancer,0.510517,0.472274,0.038243


#### d) Freedom versus equality

In [11]:
print("Female Authors: {:.2f}".format(cohen_d(freedom, equality, male_terms, female_terms, female_vectors)))
print("Male Authors: {:.2f}".format(cohen_d(freedom, equality, male_terms, female_terms, male_vectors)))
print("Movie Scripts: {:.2f}".format(cohen_d(freedom, equality, male_terms, female_terms, movie_vectors)))
print("Song Lyrics: {:.2f}".format(cohen_d(freedom, equality, male_terms, female_terms, lyrics_vectors)))

Female Authors: 0.79
Male Authors: 0.53
Movie Scripts: 1.33
Song Lyrics: 0.24


In add datasets, the concept of freedom is more stronly associated with male terms than with female terms, however the effect is strongest in the works of female authors and in movie scripts